In [26]:
#hide
!pip install -r ../requirements.txt 

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


# Introducing fastpages with hvplot
> An easy to use blogging platform with extra features for <a href="https://jupyter.org/">Jupyter Notebooks</a>.

- toc: true 
- badges: true
- comments: true
- sticky_rank: 1
- author: Thomas PEDOT
- image: images/diagram.png
- categories: [fastpages, jupyter]


In [27]:
#hide
from influxdb_client import InfluxDBClient

import pandas as pd
import time
import requests
import hvplot.pandas
from decouple import config
from urllib.parse import urlparse
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

TOKEN = config("TOKEN")
ORG = config("ORG")
INFLUXDB_URL = config("INFLUXDB_URL")
# parsed_uri = urlparse(INFLUXDB_URL)
# host = '{uri.scheme}://{uri.hostname}'.format(uri=parsed_uri)
# port = parsed_uri.port
# INFLUXDB_DB = config("INFLUXDB_DB")


In [28]:
#hide
client = InfluxDBClient(url=INFLUXDB_URL, token=TOKEN, org=ORG)

client.health()

{'checks': [],
 'commit': '4e7a59bb9a',
 'message': 'ready for queries and writes',
 'name': 'influxdb',
 'status': 'pass',
 'version': '2.0.4'}

In [29]:
# https://www.influxdata.com/blog/getting-started-with-influxdb-and-pandas/
query_api = client.query_api()
query = '''from(bucket: "energie")
  |> range(start: -1d)
  |> filter(fn: (r) => r["_measurement"] == "solarmanpv")
  |> filter(fn: (r) => r["_field"] == "Courant DC PV1" or r["_field"] == "Courant DC PV2")
  
  |> yield(name: "mean")'''
result = client.query_api().query_data_frame(org=ORG, query=query)
result.columns = ['result', 'table', 'start', 'stop', 'time', 'value', 'field',
       'measurement', 'host', 'topic']
result.head()

,result,table,start,stop,time,value,field,measurement,host,topic
0,mean,0,2021-04-05 20:35:45.234258+00:00,2021-04-06 20:35:45.234258+00:00,2021-04-05 20:35:51.492122+00:00,0.0,Courant DC PV1,solarmanpv,telegraf1,SolarmanPV/realTimeDataImp
1,mean,0,2021-04-05 20:35:45.234258+00:00,2021-04-06 20:35:45.234258+00:00,2021-04-05 20:36:12.418968+00:00,0.0,Courant DC PV1,solarmanpv,telegraf1,SolarmanPV/realTimeDataImp
2,mean,0,2021-04-05 20:35:45.234258+00:00,2021-04-06 20:35:45.234258+00:00,2021-04-05 20:36:35.397862+00:00,0.0,Courant DC PV1,solarmanpv,telegraf1,SolarmanPV/realTimeDataImp
3,mean,0,2021-04-05 20:35:45.234258+00:00,2021-04-06 20:35:45.234258+00:00,2021-04-05 20:36:55.406717+00:00,0.0,Courant DC PV1,solarmanpv,telegraf1,SolarmanPV/realTimeDataImp
4,mean,0,2021-04-05 20:35:45.234258+00:00,2021-04-06 20:35:45.234258+00:00,2021-04-05 20:37:16.688576+00:00,0.0,Courant DC PV1,solarmanpv,telegraf1,SolarmanPV/realTimeDataImp


In [30]:
#hide
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[("time", "@Date")])

In [31]:
#hide
# https://www.influxdata.com/blog/streaming-time-series-with-jupyter-and-influxdb/
from bokeh.models import HoverTool 
from bokeh.models.formatters import DatetimeTickFormatter
formatter = DatetimeTickFormatter(days="%m/%d %H:%M",
months="%m/%d %H:%M",
hours="%m/%d %H:%M",
minutes="%m/%d %H:%M")

influxdb_hvplot = result.hvplot(x="time", by="field", y="value", xformatter=formatter)

influxdb_hvplot.opts(tools=[hover])

:NdOverlay   [field]
   :Curve   [time]   (value)

In [32]:
#hide
from bokeh.resources import INLINE
hvplot.save(influxdb_hvplot, '../_includes/influxdb_hvplot.html')


In [33]:
#hide
!sed -i 's/{\%F \%T}/{\% raw \%}{\%F \%T}{\% endraw \%}/g' ../_includes/influxdb_hvplot.html

{% include influxdb_hvplot.html %}

In [34]:
# influxdb_hvplot